## Required packages and .ipynb

Soma_eqs.ipynb, Dendritic_eqs.ipynb and Synapses.ipynb are required for the proper functioning of this file.

In [1]:
%run "/Users/Mathilde/Documents/Modèles/Prefrontal_Wei/Cells_and_synapses/Soma_eqs.ipynb"
%run "/Users/Mathilde/Documents/Modèles/Prefrontal_Wei/Cells_and_synapses/Dendritic_eqs.ipynb"
%run "/Users/Mathilde/Documents/Modèles/Prefrontal_Wei/Cells_and_synapses/Synapses.ipynb"

In [2]:
from brian2 import *
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

## Defining a cortical layer: instantiating neurons, synapses and Poisson inputs

The function create_cortical_layer permits the generation of a single cortical layer of the prefrontal cortex. A cortical layer is composed of pyramidal (PY) and interneurons/inhibitory cells (IN).
N is considered to be the number of PY cells needed. The number of IN is logically deduced from it (N/4).

In [3]:
def create_cortical_layer(N,radii_AMPA_PYPY):
    
    ###Preferences
    prefs.codegen.target = 'cython'
    defaultclock.dt = 0.02*ms
    
    ###Parameters
    #Channel-specific conductances per unit of surface
    g_syn_ampa_pypy = 0.08*usiemens
    g_syn_nmda_pypy = 0.006*usiemens #0.006
    g_syn_ampa_pyin = 0.08*usiemens
    g_syn_nmda_pyin = 0.005*usiemens #0.005
    g_syn_gabaa_inpy = 0.25*usiemens #0.25*usiemens
    #Reverse potentials
    E_ampa = 0*mV
    E_nmda = 0*mV
    E_gabaa = -70*mV
    #Fraction of resources used per AP
    U_gabaa = 0.073
    U_ampa = 0.07
    #Rate constants
    alpha_ampa = 1.1*ms**-1*mM**-1
    beta_ampa = 0.19*kHz
    alpha_nmda = 1*ms**-1*mM**-1
    beta_nmda = 0.0067*kHz
    alpha_gabaa = 10.5*ms**-1*mM**-1
    beta_gabaa = 0.166*kHz
    #Number of neurons
    N_PY,N_IN = N,N/4
    #Fictional & arbitrary position of neurons for proper synapses.connect() condition
    neuron_spacing = 1*um
    #Amplitudes for minis
    #A_PY_PY = 0.2*usiemens
    #A_PY_IN = 0.05*usiemens
    A_PY_PY = 0.02*usiemens
    A_PY_IN = 0.05*usiemens
    
    #Areas of the different neurons
    s_Soma_PYIN = 10**-6*cm**2
    s_Dend_PY = 165*s_Soma_PYIN
    s_Dend_IN = 50*s_Soma_PYIN
    
    ###Instantiate neurons
    #Pyramidal
    PY_dendrite = NeuronGroup(N_PY,Dendritic_eqs,method='rk4',threshold='v>-10*mV',refractory=15*ms)
    PY_dendrite.v = -68*mvolt
    PY_dendrite.m_na = 0.05
    PY_dendrite.h_na = 0.95
    PY_dendrite.m_nap = 0.00
    PY_dendrite.m_km = 0.05
    PY_dendrite.m_kca = 0.00
    PY_dendrite.m_hva = 0.05
    PY_dendrite.h_hva = 0.05
    PY_dendrite.CA_i = 1e-4*uM 
    PY_dendrite.g_na = 0.8*msiemens*cm**-2 
    PY_dendrite.g_nap = 2.5*msiemens*cm**-2 
    PY_dendrite.g_km = 0.02*msiemens*cm**-2
    PY_dendrite.rho = 165
    PY_dendrite.x = 'i*neuron_spacing'

    PY_soma = NeuronGroup(N_PY,Soma_eqs,method='rk4',threshold='v>0*mV',refractory=3*ms,events={'custom_poisson_PY':'rand()<mean_rate_PY*dt','custom_poisson_IN':'rand()<mean_rate_IN*dt','custom_poisson_inter':'rand()<mean_rate_inter*dt'})
    PY_soma.h_na = 0.95
    PY_soma.m_na = 0.05
    PY_soma.m_nap = 0.00
    PY_soma.n_k = 0.05
    PY_soma.g_na = 3000*msiemens*cm**-2
    PY_soma.g_nap = 15*msiemens*cm**-2
    PY_soma.g_k = 200*msiemens*cm**-2
    PY_soma.x = 'i*neuron_spacing'
    
    #Interneurons
    IN_dendrite = NeuronGroup(N_IN,Dendritic_eqs,method='rk4',threshold='v>-10*mV',refractory=15*ms)
    IN_dendrite.v = -68*mvolt
    IN_dendrite.m_na = 0.05
    IN_dendrite.h_na = 0.95
    IN_dendrite.m_nap = 0.00
    IN_dendrite.m_km = 0.05
    IN_dendrite.m_kca = 0.00
    IN_dendrite.m_hva = 0.05
    IN_dendrite.h_hva = 0.05
    IN_dendrite.CA_i = 1e-4*uM
    IN_dendrite.g_na = 0.8*msiemens*cm**-2 
    IN_dendrite.g_nap = 0.0*msiemens*cm**-2 #differs from PY_dendrite
    IN_dendrite.g_km = 0.03*msiemens*cm**-2 #differs from PY_dendrite
    IN_dendrite.rho = 50 #differs from PY_dendrite
    IN_dendrite.x = '(4*i+1.5)*neuron_spacing'
    
    IN_soma = NeuronGroup(N_IN,Soma_eqs,method='rk4',threshold='v>0*mV',refractory=3*ms,events={'custom_poisson_PY':'rand()<mean_rate_PY*dt','custom_poisson_IN':'rand()<mean_rate_IN*dt','custom_poisson_inter':'rand()<mean_rate_inter*dt'})
    IN_soma.h_na = 0.95
    IN_soma.m_na = 0.05
    IN_soma.m_nap = 0.00
    IN_soma.n_k = 0.05
    IN_soma.g_na = 3000*msiemens*cm**-2
    IN_soma.g_nap = 0.00*msiemens*cm**-2 #differs from PY_dendrite
    IN_soma.g_k = 200*msiemens*cm**-2
    IN_soma.x = '(4*i+1.5)*neuron_spacing'
    
    ###Define gap junctions between soma and dendritic compartments for both PY and IN
    eq_gap_dendrite ='''
        Igap_post = g * (v_post - v_pre) : amp * meter**-2 (summed)
        g : siemens * meter**-2
    '''
    eq_gap_soma ='''
        vgap_post = v_pre : volt (summed)
    '''
    
    #From soma to dendrites
    gapPY_som_den = Synapses(PY_soma,PY_dendrite,model=eq_gap_dendrite)
    gapPY_som_den.connect(j = 'i')
    gapPY_som_den.g = 1 / (R * s_Dend_PY)
    
    gapIN_som_den = Synapses(IN_soma,IN_dendrite,model=eq_gap_dendrite)
    gapIN_som_den.connect(j = 'i')
    gapIN_som_den.g = 1 / (R * s_Dend_IN)
    
    #From dendrites to soma
    gapPY_den_som = Synapses(PY_dendrite,PY_soma,model=eq_gap_soma)
    gapPY_den_som.connect(j = 'i')
    
    gapIN_den_som = Synapses(IN_dendrite,IN_soma,model=eq_gap_soma)
    gapIN_den_som.connect(j = 'i')
    
    ###Instantiate synapses
    #from Bazhenov et al. 2002, connections appears to be from somas to dendrites
    S_AMPA_PY_PY = syn_ampa(PY_soma,PY_dendrite,'IsynAMPA_PY_PY',s_Dend_PY,'abs(i-j)<='+str(radii_AMPA_PYPY)+'and i!=j',g_syn_ampa_pypy,g_syn_ampa_pypy,E_ampa,alpha_ampa,beta_ampa,U_ampa,A_PY_PY,'IEPSPs_PY_PY',0)
    S_AMPA_PY_PY.t_last_spike = -100*ms
    S_AMPA_PY_PY.t_last_spike_Poisson_PY = -10*ms
    S_AMPA_PY_PY.t_last_spike_Poisson_IN = -10*ms
    S_AMPA_PY_PY.D = 1
    S_AMPA_PY_PY.A_mEPSP = A_PY_PY
    S_AMPA_PY_PY.w = 0
    S_AMPA_PY_IN = syn_ampa(PY_soma,IN_dendrite,'IsynAMPA_PY_IN',s_Dend_IN,'abs(x_post-x_pre) <= 8.4*um',g_syn_ampa_pyin,g_syn_ampa_pyin,E_ampa,alpha_ampa,beta_ampa,U_ampa,A_PY_IN,'IEPSPs_PY_IN',1)
    S_AMPA_PY_IN.t_last_spike = -100*ms
    S_AMPA_PY_IN.t_last_spike_Poisson_PY = -10*ms
    S_AMPA_PY_IN.t_last_spike_Poisson_IN = -10*ms
    S_AMPA_PY_IN.D = 1
    S_AMPA_PY_IN.A_mEPSP = A_PY_IN
    S_AMPA_PY_IN.w = 0
    S_NMDA_PY_PY = syn_nmda(PY_soma,PY_dendrite,'IsynNMDA_PY_PY',s_Dend_PY,'abs(i-j)<=5 and i!=j',g_syn_nmda_pypy,E_nmda,alpha_nmda,beta_nmda) #not specified in the paper, took the same as AMPA (from info on Bazhenov et al.,2002)
    S_NMDA_PY_PY.t_last_spike = -100*ms
    S_NMDA_PY_IN = syn_nmda(PY_soma,IN_dendrite,'IsynNMDA_PY_IN',s_Dend_IN,'abs(x_post-x_pre) <= 8.4*um',g_syn_nmda_pyin,E_nmda,alpha_nmda,beta_nmda) #same
    S_NMDA_PY_IN.t_last_spike = -100*ms
    S_GABAA_IN_PY = syn_gabaa(IN_soma,PY_dendrite,'IsynGABAA_IN_PY',s_Dend_PY,'abs(x_post-x_pre) <= 4.6*um',g_syn_gabaa_inpy,E_gabaa,alpha_gabaa,beta_gabaa,U_gabaa)
    S_GABAA_IN_PY.t_last_spike = -100*ms
    S_GABAA_IN_PY.D = 1
    
    ###Define monitors
    #General neuron monitoring
    #V1=StateMonitor(PY_dendrite,('v','I_nap','I_kca','I_hva','I_km','CA_i','I_kl','I_l','I_na'),record=True)
    #V2=StateMonitor(PY_soma,('v','I_nap','I_k','I_na','mean_rate_PY','t_last_spike_PY'),record=True)
    #V3=StateMonitor(IN_dendrite,('v','I_nap','I_kca','I_hva','I_km','CA_i','I_kl','I_l','I_na'),record=True)
    #V4=StateMonitor(IN_soma,('v','I_nap','I_k','I_na'),record=True)   
    V1=StateMonitor(PY_dendrite,('v'),record=True)
    V2=StateMonitor(PY_soma,('v'),record=True)
    V3=StateMonitor(IN_dendrite,('v'),record=True)
    V4=StateMonitor(IN_soma,('v'),record=True) 
    
    #Spike monitoring
    #R1=SpikeMonitor(PY_dendrite,record=True)
    R2=SpikeMonitor(PY_soma,record=True)
    #R3=SpikeMonitor(IN_dendrite,record=True)
    R4=SpikeMonitor(IN_soma,record=True)
    
    #Synaptic currents monitoring
    #I1=StateMonitor(PY_dendrite,('Isyn','IEPSPs_PY_PY'),record=True)
    #I1=StateMonitor(PY_dendrite,('Isyn','IEPSPs_PY_PY','IEPSPs_L_V_L_VI','IEPSPs_L_V_L_II','IEPSPs_L_II_L_V','IEPSPs_L_II_L_VI','IEPSPs_L_VI_L_V','IEPSPs_L_VI_L_II'),record=True)
    #I2=StateMonitor(IN_dendrite,('Isyn','IEPSPs_PY_IN'),record=True)
    I1=StateMonitor(PY_dendrite,('Iext'),record=True)
    I2=StateMonitor(IN_dendrite,'Iext',record=True)
    
    #Synapses monitoring
    #S1=StateMonitor(S_AMPA_PY_PY,('w','g_syn','t_last_spike','apost','apre','D','t_last_spike_Poisson_PY','A_mEPSP'),record=True)
    #S2=StateMonitor(S_AMPA_PY_IN,('W'),record=True)
    #S3=StateMonitor(S_NMDA_PY_PY,('W'),record=True)
    #S4=StateMonitor(S_NMDA_PY_IN,('W'),record=True)
    #S5=StateMonitor(S_GABAA_IN_PY,('W'),record=True)
    
    #miniEPSPs monitoring
    M0=EventMonitor(PY_soma,'custom_poisson_PY')
    M1=EventMonitor(PY_soma,'custom_poisson_IN')
    M2=EventMonitor(PY_soma,'custom_poisson_inter')
    
    all_neurons=PY_dendrite,PY_soma,IN_dendrite,IN_soma
    all_gap_junctions=gapPY_som_den,gapPY_den_som,gapIN_som_den,gapIN_den_som
    all_synapses=S_AMPA_PY_PY,S_AMPA_PY_IN,S_NMDA_PY_PY,S_NMDA_PY_IN,S_GABAA_IN_PY
    all_monitors=V1,V2,V3,V4,R2,R4,I1,I2,M0,M1,M2
    
    return all_neurons,all_synapses,all_gap_junctions,all_monitors  


## Creation of the network, and simulation

## Plotting results